In [67]:
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

In [70]:

from pyecharts.charts import Bar
from pyecharts import options as opts
from pyecharts.globals import ThemeType

bar =(
    Bar(init_opts=opts.InitOpts(theme=ThemeType.WESTEROS,width="100%", height="400px"))
    .add_xaxis(["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"])
    .add_yaxis("商家A",[5, 20, 36,10, 75, 90])
    .add_yaxis("商家B",[15,6,45, 20,35, 66])
    .set_global_opts(
        title_opts=opts.TitleOpts(title="主标题",subtitle="副标题"),
                        datazoom_opts=[
                            opts.DataZoomOpts(type_="slider"),     # 底部滑块
                            opts.DataZoomOpts(type_="inside")      # 鼠标滚轮缩放（内嵌）
                        ],
                    )
    )

bar.load_javascript()

# print("lll")

In [84]:
import pandas as pd
import io
import csv

from pyecharts.charts import Line
from pyecharts import options as opts
from pyecharts.globals import RenderType
from IPython.display import display, HTML
import ipywidgets as widgets

# def styled_table(df):
#     return HTML(f"""
#     <div style="overflow-x: auto; width: 100%;">
#         {df.to_html(index=False)}
#     </div>
#     """)

# 全局变量
data_raw = pd.DataFrame()
data_df = pd.DataFrame()

# 上传按钮
upload_button = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,
    description='上传表格',
    button_style='info'
)

# 标题行 / 结束行选择
start_row_dropdown = widgets.Dropdown(description="标题行：")
end_row_dropdown = widgets.Dropdown(description="结束行：")

# 网格透明度滑块
opacity_slider = widgets.FloatSlider(
    value=0.8,
    min=0.0,
    max=1.0,
    step=0.1,
    description='网格透明度',
    readout_format='.1f',
    continuous_update=False
)

line_width_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=10.0,
    step=0.5,
    description='线条粗细',
    readout_format='.1f',
    continuous_update=False
)

# 最大/最小值和平均值 checkbox 控件
show_avg_cb = widgets.Checkbox(value=False, description='显示平均值')
show_extremes_cb = widgets.Checkbox(value=False, description='显示最大/最小值')

style_row_controls = widgets.HBox([
    opacity_slider,
    line_width_slider,
    show_avg_cb,
    show_extremes_cb
])

# X/Y轴 checkbox 控件容器
x_checkboxes = []
y_checkboxes = []

x_box = widgets.HBox()
y_box = widgets.HBox()

# 输出区域
output_chart = widgets.Output()

# 宽容 CSV 读取函数
def read_csv_loose(content_bytes):
    try:
        decoded_text = content_bytes.decode('utf-8')
    except UnicodeDecodeError:
        decoded_text = content_bytes.decode('gbk', errors='ignore')

    reader = csv.reader(io.StringIO(decoded_text))
    rows = list(reader)
    max_cols = max(len(r) for r in rows)
    normalized = [r + [''] * (max_cols - len(r)) for r in rows]
    return pd.DataFrame(normalized)

# 上传文件响应
def on_upload_change(change):
    global data_raw
    output_chart.clear_output()

    if upload_button.value:
        try:
            if isinstance(upload_button.value, dict):  # ipywidgets 7.x
                uploaded_file = next(iter(upload_button.value.values()))
                filename = list(upload_button.value.keys())[0]
                content = uploaded_file['content']
            else:  # ipywidgets 8.x
                uploaded_file = upload_button.value[0]
                filename = uploaded_file['name']
                content = uploaded_file['content']
        except Exception as e:
            with output_chart:
                print("无法读取上传内容：", e)
            return

        try:
            if filename.endswith('.csv'):
                if isinstance(content, memoryview):
                    content = content.tobytes()
                data_raw = read_csv_loose(content)
            elif filename.endswith('.xlsx'):
                data_raw = pd.read_excel(io.BytesIO(content), header=None, engine='openpyxl')
            else:
                with output_chart:
                    print("不支持的文件格式")
                return
        except Exception as e:
            with output_chart:
                print("读取失败:", e)
            return

        # 更新行选择器
        row_count = len(data_raw)
        options = [(f"第 {i+1} 行", i) for i in range(row_count)]
        start_row_dropdown.options = options
        start_row_dropdown.value = 0
        end_row_dropdown.options = options
        end_row_dropdown.value = row_count - 1

        with output_chart:
            print("文件读取成功，共", row_count, "行")

# 构建 checkbox 控件
def build_checkboxes(headers):
    global x_checkboxes, y_checkboxes

    def on_x_change(change):
        if change['new']:
            for cb in x_checkboxes:
                if cb is not change['owner']:
                    cb.value = False
        update_chart()

    def on_y_change(change):
        update_chart()

    x_checkboxes = []
    y_checkboxes = []

    for h in headers:
        x_cb = widgets.Checkbox(description=h, value=False, indent=False)
        x_cb.observe(on_x_change, names='value')
        x_checkboxes.append(x_cb)

        y_cb = widgets.Checkbox(description=h, value=False, indent=False)
        y_cb.observe(on_y_change, names='value')
        y_checkboxes.append(y_cb)

    x_box.children = x_checkboxes
    y_box.children = y_checkboxes

# 更新列选择器
def update_column_selectors(*args):
    global data_df

    output_chart.clear_output()
    try:
        header_index = start_row_dropdown.value
        end_index = end_row_dropdown.value

        if header_index is None or end_index is None or header_index >= end_index:
            with output_chart:
                print("标题行设置错误")
            return

        headers = data_raw.iloc[header_index].tolist()
        data_df = data_raw.iloc[header_index + 1: end_index + 1].copy()
        data_df.columns = headers
        data_df.reset_index(drop=True, inplace=True)

        print(f"数据预览：{headers}")
        build_checkboxes(headers)

        update_chart()

        # if header_index is not None and end_index is not None and header_index < end_index:
        print(f"✔ 当前标题行为第 {header_index + 1} 行，结束行为第 {end_index + 1} 行")

        print("\n📌 表头下的前 10 行预览：")
        preview_start = data_raw.iloc[header_index + 1: header_index + 11]
        preview_start.columns = data_raw.iloc[header_index]
        display(preview_start.reset_index(drop=True))

        print("\n📌 结束行前的最后 10 行预览：")
        preview_end = data_raw.iloc[max(header_index + 1, end_index - 9): end_index + 1]
        preview_end.columns = data_raw.iloc[header_index]
        display(preview_end.reset_index(drop=True))


        # print(f"✔ 当前标题行为第 {header_index + 1} 行，结束行为第 {end_index + 1} 行")

        # print("\n📌 表头下的前 10 行预览：")
        # preview_start = data_raw.iloc[header_index + 1: header_index + 11]
        # preview_start.columns = data_raw.iloc[header_index]
        # display(styled_table(preview_start.reset_index(drop=True)))

        # print("\n📌 结束行前的最后 10 行预览：")
        # preview_end = data_raw.iloc[max(header_index + 1, end_index - 9): end_index + 1]
        # preview_end.columns = data_raw.iloc[header_index]
        # display(styled_table(preview_end.reset_index(drop=True)))


    except Exception as e:
        with output_chart:
            print("更新列选择器失败：", e)

# 绘制图表
def update_chart(*args):
    output_chart.clear_output()

    if data_df.empty:
        return

    x_vals = [cb.description for cb in x_checkboxes if cb.value]
    y_cols = [cb.description for cb in y_checkboxes if cb.value]

    if len(x_vals) != 1 or not y_cols:
        with output_chart:
            if len(x_vals) != 1:
                print("请仅选择一个 X 轴字段")
            elif not y_cols:
                print("请至少选择一个 Y 轴字段")
        return

    x_col = x_vals[0]

    try:
        x_data = data_df[x_col].astype(str).tolist()
        line = Line(init_opts=opts.InitOpts(width="100%", height="600px",renderer=RenderType.CANVAS)).add_xaxis(x_data)

        for y in y_cols:
            y_data = pd.to_numeric(data_df[y], errors='coerce').fillna(0).tolist()
            # line.add_yaxis(y, y_data)
            line.add_yaxis(
                series_name=y,
                y_axis=y_data,
                # markline_opts=opts.MarkLineOpts(data=[opts.MarkLineItem(type_="average")]),
                markline_opts=opts.MarkLineOpts(
                    data=[
                        opts.MarkLineItem(type_="average")
                    ] if show_avg_cb.value else None
                ),
                markpoint_opts=opts.MarkPointOpts(
                    data=[
                        opts.MarkPointItem(type_="max"),
                        opts.MarkPointItem(type_="min")
                    ] if show_extremes_cb.value else None
                ),
                linestyle_opts=opts.LineStyleOpts(width=line_width_slider.value),
            )


        line.set_global_opts(
            title_opts=opts.TitleOpts(title="曲线图"),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            toolbox_opts=opts.ToolboxOpts(),
            datazoom_opts=[
                opts.DataZoomOpts(type_="slider"),
                opts.DataZoomOpts(type_="inside")
            ],
            # xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=0.8))),
            # yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=0.8)))
            xaxis_opts=opts.AxisOpts(
                splitline_opts=opts.SplitLineOpts(
                    is_show=True,
                    linestyle_opts=opts.LineStyleOpts(opacity=opacity_slider.value)
                )
            ),
            yaxis_opts=opts.AxisOpts(
                splitline_opts=opts.SplitLineOpts(
                    is_show=True,
                    linestyle_opts=opts.LineStyleOpts(opacity=opacity_slider.value)
                )
            )
        )

        with output_chart:
            display(line.render_notebook())


    except Exception as e:
        with output_chart:
            print("绘图失败：", e)

opacity_slider.observe(update_chart, names='value')
line_width_slider.observe(update_chart, names='value')
start_row_dropdown.observe(update_column_selectors, names='value')
end_row_dropdown.observe(update_column_selectors, names='value')
upload_button.observe(on_upload_change, names='value')
show_avg_cb.observe(update_chart, names='value')
show_extremes_cb.observe(update_chart, names='value')

# 显示控件布局
axis_selector_box = widgets.VBox([
    widgets.HTML("<b>字段选择</b>"),
    widgets.HTML("<b>X 轴：</b>"),
    x_box,
    widgets.HTML("<b>Y 轴：</b>"),
    y_box
])

# 最终界面布局
# display()
display(widgets.HBox([upload_button, start_row_dropdown, end_row_dropdown]))

header_index = start_row_dropdown.value
end_index = end_row_dropdown.value


,﻿通道号,托盘条码,电芯条码,工步号,工步类型,结束状态,工步运行时间,开始温度 ℃,结束温度 ℃,平均温度 ℃,...,,,,,,,,,,
0,2,JHM01C07HCJSA00109,0TICBC7DFLB121E990011163,1,静置,(下)以时间条件结束,00:02:00,50.8|52.9,50.7|52.9,51.838842975206695,...,,,,,,,,,,
1,2,JHM01C07HCJSA00109,0TICBC7DFLB121E990011163,2,恒流充电,(下)以电压条件结束,00:13:29,50.7|52.9,50.6|52.9,51.79093711467289,...,,,,,,,,,,
2,2,JHM01C07HCJSA00109,0TICBC7DFLB121E990011163,2,恒流充电,(下)用户暂停,00:13:52,51.1|53.4,51.1|53.5,52.26874999999998,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,通道号,工步号,工步类型,通道切入状态,工步时间,电脑时间,负压值kPa,电芯电压mV,电流mA,容量mAh,...,主端口电压mV,回路阻抗mΩ,拘束力,左库位温度1℃,右库位温度2℃,前库位温度3℃,后库位温度4℃,电芯温度℃,电芯温度2℃,事件信息
5,2,1,静置,cc切入,00:00:00,09/13/2024 16:45:39,-25.3,3071.55,0,0,...,51930.94,0,0,52.1,52.2,52.8,52.9,50.8|52.9,52.9,
6,2,1,静置,cc切入,00:00:01,09/13/2024 16:45:40,-26,3071.55,0,0,...,51880.58,0,0,52,52.2,52.8,52.9,50.8|52.9,52.9,
7,2,1,静置,cc切入,00:00:02,09/13/2024 16:45:41,-26.8,3071.55,0,0,...,51807.58,0,0,52,52.2,52.8,52.9,50.7|52.9,52.9,
8,2,1,静置,cc切入,00:00:03,09/13/2024 16:45:42,-27.4,3071.55,0,0,...,51735.7,0,0,52,52.1,52.8,52.9,50.8|52.9,52.9,
9,2,1,静置,cc切入,00:00:04,09/13/2024 16:45:43,-28.5,3071.55,0,0,...,51665.38,0,0,52,52.2,52.7,52.9,50.8|52.9,52.9,


,﻿通道号,托盘条码,电芯条码,工步号,工步类型,结束状态,工步运行时间,开始温度 ℃,结束温度 ℃,平均温度 ℃,...,,,,,,,,,,
0,0,是,过充电流上限,,25000,mA,,,,,...,,,,,,,,,,
1,0,是,接触不良电压上限,,100000,mV,,,,,...,,,,,,,,,,
2,0,是,极耳不良电压上限,,3700,mV,,,,,...,,,,,,,,,,
3,0,是,接触阻抗保护上限,,2,mΩ,,,,,...,,,,,,,,,,
4,0,是,回路阻抗保护上限,,8,mΩ,,,,,...,,,,,,,,,,
5,0,是,急停电压上限(V08),,3850,mV,,,,,...,,,,,,,,,,
6,0,是,闲时累加电压上限(V09),,180,mV,,,,,...,,,,,,,,,,
7,0,是,闲时累加电压上限延时时间(V09),,300,ms,,,,,...,,,,,,,,,,
8,0,否,,,,,T05&&T06&&S01=A01&&A02&&A04&&A05&&A06&&A07&&A09,,,,...,,,,,,,,,,
9,,,,,,,,,,,...,,,,,,,,,,


In [81]:
display(widgets.HBox([opacity_slider, line_width_slider, show_avg_cb, show_extremes_cb]))
display(axis_selector_box)


In [83]:
display(widgets.GridBox([output_chart]))

GridBox(children=(Output(),))